In [ ]:
# For Colab users uncomment the following lines to install required packages
# !pip install lm_eval  langdetect -q
# !pip install git+https://github.com/felipemaiapolo/tinyBenchmarks

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 21.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 111.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 kB 9.9 MB/s eta 0:00:00
  Cloning https://github.com/felipemaiapolo/tinyBenchmarks to /tmp/pip-req-build-q06c477h
  Running command git clone --filter=blob:none --quiet https://github.com/felipemaiapolo/tinyBenchmarks /tmp/pip-req-build-q06c477h
  Resolved https://github.com/felipemaiapolo/tinyBenchmar

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from lm_eval import evaluator
from joblib import dump
import wandb
import numpy as np

In [ ]:
# Login to Weights & Biases
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dannyai (dannyai-danny-the-analyst) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
# Create W&B and run
wandb.init(project="phi4_african_history", name="phi4_african_history_lora_axolotl_bench_mark")

In [ ]:
# Define your IDs
base_model_id = "microsoft/Phi-4-mini-instruct"
lora_adapter_id = "DannyAI/phi4_lora_axolotl" # Your HF path

In [ ]:
config = {
    # "model": lora_model,
    "tasks": [
              "tinyTruthfulQA",
              "tinyMMLU"
              ],
    "batch_size": 4,
}

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

results = evaluator.simple_evaluate(
    model="hf",
    model_args=f"pretrained={base_model_id},peft={lora_adapter_id},parallelize=True,trust_remote_code=False",
    tasks=config["tasks"],
    device=device,
    batch_size=config["batch_size"],
)


dump(results, "evaluation_results.joblib")

In [6]:
results['results']

{'tinyMMLU': {'alias': 'tinyMMLU',
  'acc_norm,none': np.float64(0.6737149983441648),
  'acc_norm_stderr,none': 'N/A'},
 'tinyTruthfulQA': {'alias': 'tinyTruthfulQA',
  'acc,none': np.float64(0.46625508603749494),
  'acc_stderr,none': 'N/A'}}

In [ ]:
# Clean and flatten the dictionary
summary_metrics = {}

for task, metrics in results['results'].items():
    for metric_name, value in metrics.items():
        # Only log numeric values, skip 'alias' and 'N/A'
        if isinstance(value, (int, float, np.float64)):
            # Create a clean name, e.g., "tinyMMLU/acc_norm"
            clean_name = f"{task}/{metric_name.replace(',none', '')}"
            summary_metrics[clean_name] = float(value)

# Log onto active W&B run
wandb.log(summary_metrics)

# Finish & sync data immediately
wandb.finish()
print("Results successfully synced to W&B!")

tinyMMLU/acc_norm,▁
tinyTruthfulQA/acc,▁
tinyMMLU/acc_norm,0.67371
tinyTruthfulQA/acc,0.46626


Results successfully synced to W&B!
